In [2]:
import sympy as sym
import numpy as np

## Dynamic model

Specify the physical parameters:

In [3]:
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 1.1

Derive the equations of motion:

In [4]:
# roll, pitch, yaw angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# resultant torques
lt = sym.nsimplify(lw) * sym.sqrt(2) / 2
T1 = - tau_1 * sym.Matrix([[lt], [0], [lt]])
T2 = - tau_2 * sym.Matrix([[-lt], [0], [lt]])
T3 = - tau_3 * sym.Matrix([[0], [lt], [lt]])
T4 = - tau_4 * sym.Matrix([[0], [-lt], [lt]])
T = T1 + T2 + T3 + T4

# parameters
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# rotation matrices
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry * Rz).T * ex, Rz.T * ey, ez).inv(), full=True)

# euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

The equations of motion have this form:

$$\begin{bmatrix} \dot{\phi} \\ \dot{\theta} \\ \dot{\psi} \\ \dot{w_x} \\ \dot{w_y} \\ \dot{w_z} \end{bmatrix} = f\left(\phi, \theta, \psi, w_x, w_y, w_z, \tau_1, \tau_2, \tau_3, \tau_4\right)$$

Here is the function $f$:

In [5]:
f

Matrix([
[                        (w_x*cos(psi) - w_y*sin(psi))/cos(theta)],
[                                     w_x*sin(psi) + w_y*cos(psi)],
[        -w_x*cos(psi)*tan(theta) + w_y*sin(psi)*tan(theta) + w_z],
[-55*sqrt(2)*tau_1/1484 + 55*sqrt(2)*tau_2/1484 - 150*w_y*w_z/371],
[-55*sqrt(2)*tau_3/1484 + 55*sqrt(2)*tau_4/1484 + 150*w_x*w_z/371],
[                -55*sqrt(2)*(tau_1 + tau_2 + tau_3 + tau_4)/2084]])

## Sensor model

Symbolic variables for right ascension $\alpha$ and declination $\delta$ of each star:

In [6]:
alpha, delta = sym.symbols('alpha, delta')

Specify the physical parameters:

In [7]:
# Scope radius
r = 0.8 / 2.1

Derive the sensor model:

In [8]:
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rx * Ry * Rz

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

The sensor model has this form for each star:

$$\zeta = g(\phi, \theta, \psi, \alpha, \delta)$$

Here is the function $g$:

In [9]:
g

Matrix([
[21*((sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*sin(delta) - (sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*sin(alpha)*cos(delta) - sin(psi)*cos(alpha)*cos(delta)*cos(theta))/(8*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*sin(delta) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*sin(alpha)*cos(delta) + cos(alpha)*cos(delta)*cos(psi)*cos(theta)))],
[                                                                      21*(-sin(alpha)*sin(phi)*cos(delta)*cos(theta) + sin(delta)*cos(phi)*cos(theta) + sin(theta)*cos(alpha)*cos(delta))/(8*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*sin(delta) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*sin(alpha)*cos(delta) + cos(alpha)*cos(delta)*cos(psi)*cos(theta)))]])